In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glaucoma-detection/glaucoma.csv
/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Validation/Glaucoma_Negative/537.jpg
/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Validation/Glaucoma_Negative/521.jpg
/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Validation/Glaucoma_Negative/539.jpg
/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Validation/Glaucoma_Negative/501.jpg
/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Validation/Glaucoma_Negative/544.jpg
/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Validation/Glaucoma_Negative/489.jpg
/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Validation/Glaucoma_Negative/576.jpg
/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Validation/Glaucoma_Negative/503.jpg
/kaggle/input/glaucoma-detection/Fundus_Train_Val_

In [4]:
# --- 0. 导入必要的库 ---
import warnings
warnings.filterwarnings("ignore")

import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from glob import glob
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score

from skimage import exposure, util
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.measure import label, regionprops
from skimage.morphology import binary_opening, binary_closing, square
import scipy.io as sio # 用于加载 .mat 文件

# --- 1. UNet 类定义 ---
class UNet(nn.Module): 
    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()
        features = init_features
        self.encoder1 = self._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = self._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = self._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = self._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = self._block(features * 8, features * 16, name="bottleneck")

        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = self._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = self._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = self._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = self._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )
        self.sigmoid = nn.Sigmoid()

    def _block(self, in_channels, features, name):
        return nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=features,
                kernel_size=3,
                padding=1,
                bias=False,
            ),
            nn.BatchNorm2d(num_features=features),
            nn.ReLU(inplace=True),
            nn.Conv2d(
                in_channels=features,
                out_channels=features,
                kernel_size=3,
                padding=1,
                bias=False,
            ),
            nn.BatchNorm2d(num_features=features),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        
        logits = self.conv(dec1)
        return self.sigmoid(logits)


# --- 2. 图像预处理函数 ---
def get_bounding_box(binary_mask):
    binary = binary_mask > 0
    if not np.any(binary):
        height, width = binary_mask.shape
        return 0, 0, width, height
    labeled_image = label(binary)
    regions = regionprops(labeled_image)
    largest_region = max(regions, key=lambda r: r.area)
    min_row, min_col, max_row, max_col = largest_region.bbox
    return min_col, min_row, max_col - min_col, max_row - min_row

def gamma_correct(img, gamma=0.4):
    img_float = util.img_as_float(img)
    corrected = exposure.adjust_gamma(img_float, gamma=gamma)
    return util.img_as_ubyte(corrected)

def enhance_contrast_clahe(image, clip_limit=0.02, kernel_size=8):
    img_float = util.img_as_float(image)
    enhanced = exposure.equalize_adapthist(
        img_float,
        kernel_size=kernel_size,
        clip_limit=clip_limit
    )
    return util.img_as_ubyte(enhanced)


def segment_optic_disc_from_mat(image_path, mat_dir):
    image_stem = Path(image_path).stem
    mat_path = os.path.join(mat_dir, f"{image_stem}.mat")
    mat_data = sio.loadmat(mat_path)
    mask = mat_data['mask']
    optic_disc_mask = (mask > 0).astype(np.uint8) * 255
    return optic_disc_mask


def segment_optic_disc_by_clustering(gray_image):
    pixel_values = gray_image.reshape(-1, 1).astype(np.float32)
    kmeans = KMeans(n_clusters=6, n_init=3, random_state=0).fit(pixel_values)
    cluster_labels = kmeans.labels_.reshape(gray_image.shape)
    cluster_brightness_means = []
    for cluster_id in range(6):
        pixels_in_cluster = pixel_values[cluster_labels.ravel() == cluster_id]
        mean_brightness = pixels_in_cluster.mean() if len(pixels_in_cluster) > 0 else -np.inf
        cluster_brightness_means.append(mean_brightness)
    brightest_cluster_ids = np.argsort(cluster_brightness_means)[-2:]
    optic_disc_mask = np.isin(cluster_labels, brightest_cluster_ids)
    return (optic_disc_mask * 255).astype(np.uint8)


def segment_optic_disc_with_unet(image_path, unet_model):
    pil_image = Image.open(image_path).convert('RGB')
    original_img = np.array(pil_image)
    img_tensor = util.img_as_float(original_img)
    img_tensor = resize(img_tensor, (384, 384), anti_aliasing=True)
    img_tensor = torch.FloatTensor(img_tensor).permute(2, 0, 1).unsqueeze(0)  
    # 从模型对象获取其所在的设备
    model_device = next(unet_model.parameters()).device
    img_tensor = img_tensor.to(model_device) #
    with torch.no_grad():
        output = unet_model(img_tensor)
        mask_pred = output.squeeze().cpu().numpy()
        mask_pred = (mask_pred > 0.5).astype(np.uint8) 
    mask_pred = resize(mask_pred, original_img.shape[:2], anti_aliasing=False, order=0)
    mask_pred = (mask_pred > 0.5).astype(np.uint8) * 255
    return mask_pred


def crop_fundus_region(original_image, threshold=10):
    if original_image.ndim == 2:
        original_image = np.stack([original_image] * 3, axis=-1)
    gray_image = util.img_as_ubyte(rgb2gray(original_image))
    if np.max(gray_image) <= threshold:
        foreground_mask = np.ones_like(gray_image, dtype=np.uint8) * 255
    else:
        foreground_mask = (gray_image > threshold).astype(np.uint8) * 255
    min_col, min_row, width, height = get_bounding_box(foreground_mask)
    margin = min(100, width // 4, height // 4)
    col_start = max(0, min_col + margin)
    row_start = max(0, min_row + margin)
    col_end = min(original_image.shape[1], min_col + width - margin)
    row_end = min(original_image.shape[0], min_row + height - margin)
    if col_end <= col_start or row_end <= row_start:
        col_start, row_start, col_end, row_end = 0, 0, original_image.shape[1], original_image.shape[0]
    cropped_rgb = original_image[row_start:row_end, col_start:col_end]
    cropped_gray = gray_image[row_start:row_end, col_start:col_end]
    return cropped_rgb, cropped_gray


def refine_mask_with_morphology(binary_mask):
    binary = binary_mask > 0
    opened = binary_opening(binary, footprint=square(5))
    closed = binary_closing(opened, footprint=square(5))
    return (closed * 255).astype(np.uint8)


def crop_optic_disc_region(rgb_image, optic_disc_mask):
    min_col, min_row, width, height = get_bounding_box(optic_disc_mask)
    scale_x = rgb_image.shape[1] / optic_disc_mask.shape[1]
    scale_y = rgb_image.shape[0] / optic_disc_mask.shape[0]
    min_col = int(min_col * scale_x)
    width = int(width * scale_x)
    min_row = int(min_row * scale_y)
    height = int(height * scale_y)
    center_x = min_col + width // 2
    center_y = min_row + height // 2
    half_crop_size = 250  # 裁剪 500x500 区域
    col_start = max(0, center_x - half_crop_size)
    row_start = max(0, center_y - half_crop_size)
    col_end = min(rgb_image.shape[1], center_x + half_crop_size)
    row_end = min(rgb_image.shape[0], center_y + half_crop_size)
    if col_end <= col_start or row_end <= row_start:
        col_start, row_start = 0, 0
        col_end = min(rgb_image.shape[1], 500)
        row_end = min(rgb_image.shape[0], 500)
    optic_disc_crop = rgb_image[row_start:row_end, col_start:col_end]
    final_optic_disc_crop = util.img_as_ubyte(
        resize(optic_disc_crop, (384, 384), anti_aliasing=True)
    )
    return final_optic_disc_crop



def preprocess_fundus_image(image_path, method='unet', mat_dir=None, unet_model=None, debug=False):
    pil_image = Image.open(image_path).convert("RGB")
    original_image = np.array(pil_image)
    cropped_rgb, cropped_gray = crop_fundus_region(original_image, threshold=10)
    enhanced_gray = gamma_correct(cropped_gray, gamma=0.4)
    enhanced_gray = enhance_contrast_clahe(enhanced_gray)
    if method == 'clustering':
        optic_disc_mask = segment_optic_disc_by_clustering(enhanced_gray)
    elif method == 'unet':
        if unet_model is None:
            raise ValueError("if method='unet', must provide unet model")
        optic_disc_mask = segment_optic_disc_with_unet(image_path, unet_model)
    elif method == 'mat':
        if mat_dir is None:
            raise ValueError("if method = 'mat', must provide mat file directory")
        optic_disc_mask = segment_optic_disc_from_mat(image_path, mat_dir)
    else:
        raise ValueError(f"unkonwn method: {method}")
    refined_mask = refine_mask_with_morphology(optic_disc_mask)
    enhanced_rgb = enhance_contrast_clahe(cropped_rgb)
    final_crop = crop_optic_disc_region(enhanced_rgb, refined_mask)
    
    if debug:
        return {
            "original_image": original_image,
            "cropped_fundus": cropped_rgb,
            "optic_disc_mask": refined_mask,
            "final_optic_disc_crop": final_crop
        }
    else:
        return final_crop

# --- 4. 定义加载模型的函数 ---
def load_model(model_class, model_save_path, in_channels=3, out_channels=1, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"device:{device}")

    model = model_class(in_channels=in_channels, out_channels=out_channels).to(device)
    model.eval()  

    try:
        with torch.serialization.safe_globals([np.dtype, np.core.multiarray.scalar]):
            checkpoint = torch.load(model_save_path, map_location=device, weights_only=True)
    except Exception as e:
        print(f"weights_only=True load failed: {e}")
        print("try weights_only=False...")
        checkpoint = torch.load(model_save_path, map_location=device, weights_only=False)

    model.load_state_dict(checkpoint['model_state_dict'])

    print(f"success, device:{device}")
    return model

# --- 5. 批量预处理函数 ---
def batch_preprocess_images(
    input_directory,
    output_directory,
    method='unet',
    unet_model=None,
    image_extension="*.jpg",
    save_quality=95
):
    os.makedirs(output_directory, exist_ok=True)
    print(f"output directory:{output_directory}")

    image_paths = sorted(glob(os.path.join(input_directory, image_extension)))
    if not image_paths:
        print(f"no {image_extension} file in {input_directory}")
        return
    # imag_paths = image_paths[:10]

    for idx, image_path in enumerate(image_paths, 1):
        filename = Path(image_path).name
        print(f" ({idx}/{len(image_paths)}) dealing {filename}")

        # 调用你的预处理函数（确保它已定义）
        final_crop = preprocess_fundus_image(
            image_path=image_path,
            method=method,
            unet_model=unet_model if method == 'unet' else None)

        output_filename = f"{Path(image_path).stem}_preprocessed.jpg"
        output_path = os.path.join(output_directory, output_filename)

        # 保存图像
        img_to_save = Image.fromarray(final_crop)
        img_to_save.save(output_path, quality=save_quality)

        print(f" saved in {output_path}")

In [ ]:
model = load_model(
    model_class=UNet,
    model_save_path="/kaggle/input/best-unet/pytorch/default/1/best_unet_optic_disc.pth",
    in_channels=3,
    out_channels=1
)

# 2. 批量预处理
batch_preprocess_images(
    input_directory="/kaggle/input/glaucoma-detection/ORIGA/ORIGA/Images/",
    output_directory="/kaggle/working/preprocessed_optic_discs_unet/",
    method='unet',
    unet_model=model,
    image_extension="*.jpg",
    save_quality=95
)

device:cpu
weights_only=True load failed: Weights only load failed. In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
Please file an issue with the following so that we can make `weights_only=True` compatible with your use case: WeightsUnpickler error: Can only build Tensor, Parameter, OrderedDict or types allowlisted via `add_safe_globals`, but got <class 'numpy.dtypes.Float64DType'>

Check the documentation of torch.load to learn more about types accepted by default with weights_only https://pytorch.org/docs/stable/generated/torch.load.html.
try weights_only=False...
success, device:cpu
output directory:/kaggle/working/preprocessed_optic_discs_unet/
 (1/650) dealing 001.jpg
 saved in /kaggle/working/preprocessed_optic_discs_unet/001_preproce